In [ ]:
!pip install accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 15.5 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 56.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import json
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, GenerationConfig
import torch
import faiss
from sentence_transformers import SentenceTransformer

# Чанкинг CSV
def clean_text(text):
    text = re.sub(r'\s+', ' ', text.replace('\xa0', ' ')).strip()
    return text

# Токенизатор для подсчета токенов
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
def token_length(text):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    return len(tokens)

# Загрузка CSV
df = pd.read_csv('/content/documentation.csv')

# Сплиттер
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512, chunk_overlap=50, length_function=token_length, separators=["\n\n", "\n", ".", " "]
)

# Чанкинг
chunks = []
for idx, row in df.iterrows():
    subsection = row['subsection']
    source = row['source']
    content = clean_text(row['content'])
    split_texts = text_splitter.split_text(content)
    for i, chunk in enumerate(split_texts):
        chunk_id = f"{idx}_{i}"
        chunks.append({
            "chunk_id": chunk_id,
            "subsection": subsection,
            "source": source,
            "content": chunk
        })

# Сохранение чанков
with open('/content/chunks.json', 'w', encoding='utf-8') as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

print(f"Создано {len(chunks)} чанков. Сохранено в /content/chunks.json")

# Эмбеддинги с rubert-base-cased-sentence
retriever_model = SentenceTransformer('DeepPavlov/rubert-base-cased-sentence').to('cuda')
texts = [chunk['content'] for chunk in chunks]
print("Генерируем эмбеддинги с ruBERT...")
embeddings = retriever_model.encode(texts, batch_size=32, show_progress_bar=True, convert_to_numpy=True).astype('float32')

# Проверка эмбеддингов
print(f"\nПроверка эмбеддингов:")
print(f"Количество: {len(embeddings)}")
print(f"Размерность: {embeddings.shape[1]}")
for i in range(min(3, len(embeddings))):
    print(f"Чанк {i+1} ({chunks[i]['subsection']}): Первые 5 элементов: {embeddings[i][:5]}")

# Сохранение
np.save('/content/embeddings_rubert.npy', embeddings)
with open('/content/chunks_metadata.json', 'w', encoding='utf-8') as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

# Создание FAISS-индекса
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
faiss.write_index(index, '/content/faiss_index_rubert.bin')



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

Генерируем эмбеддинги с ruBERT...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]


Проверка эмбеддингов:
Количество: 79
Размерность: 768
Чанк 1 (Консоль управления vCloudDirector): Первые 5 элементов: [-0.7660884  -1.6354835   1.1291286   0.27228057 -0.87624085]
Чанк 2 (Виртуальные машины (VMs)): Первые 5 элементов: [-1.071812   -1.6254451   1.1709776  -0.00183234 -1.0489055 ]
Чанк 3 (Виртуальные машины (VMs)): Первые 5 элементов: [-0.8569852  -1.8081694   0.96947324  0.14547616 -0.9975265 ]


In [ ]:
# Модель для генерации
MODEL_NAME = "IlyaGusev/saiga_llama3_8b"
DEFAULT_SYSTEM_PROMPT = "Ты - ассистент по документации. Отвечай на вопросы кратко."
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, load_in_8bit=True, torch_dtype=torch.bfloat16, device_map="auto"
).eval()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

# Ретривер
def retrieve_chunks(query, k=2, subsection_filter=None):
    query_embedding = retriever_model.encode([query]).astype('float32')
    distances, indices = index.search(query_embedding, k)
    results = []
    for i, idx in enumerate(indices[0]):
        chunk = chunks[idx]
        if subsection_filter is None or chunk['subsection'] == subsection_filter:
            results.append({
                'distance': float(distances[0][i]),
                'chunk_id': chunk['chunk_id'],
                'subsection': chunk['subsection'],
                'source': chunk['source'],
                'content': chunk['content']
            })
    return results[:k]


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

In [ ]:
# Генерация ответа
def generate_answer(query, chunks):
    context = "\n\n".join([f"Чанк {i+1} ({chunk['subsection']}): {chunk['content']}" for i, chunk in enumerate(chunks)])
    prompt = [{
        "role": "system",
        "content": DEFAULT_SYSTEM_PROMPT
    }, {
        "role": "user",
        "content": f"Вопрос: {query}\n\nКонтекст: {context}"
    }]
    prompt_text = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)

    # Вывод и сохранение промпта
    print("\nПолный промпт:")
    print("-" * 50)
    print(prompt_text)
    print("-" * 50)
    with open('/content/prompt.txt', 'w', encoding='utf-8') as f:
        f.write(prompt_text)

    # Обрезка до 8192 токенов
    tokenized_prompt = tokenizer.encode(prompt_text, add_special_tokens=False)
    if len(tokenized_prompt) > 8192:
        prompt_text = tokenizer.decode(tokenized_prompt[:8192], skip_special_tokens=True)

    # Генерация
    data = tokenizer(prompt_text, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    answer = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    return answer

# Запрос
query = "В чем разница между обязательными и предпочтительными правилами Affinity и Anti-Affinity?"
chunks = retrieve_chunks(query, k=2, subsection_filter=None)

# Вывод чанков
print(f"\nЗапрос: {query}")
for i, chunk in enumerate(chunks):
    print(f"\nЧанк {i+1}:")
    print(f"Расстояние: {chunk['distance']:.4f}")
    print(f"Подраздел: {chunk['subsection']}")
    print(f"Источник: {chunk['source']}")
    print(f"Контент: {chunk['content'][:200]}...")

# Генерация ответа
answer = generate_answer(query, chunks)
print(f"\nОтвет: {answer}")


Запрос: В чем разница между обязательными и предпочтительными правилами Affinity и Anti-Affinity?

Чанк 1:
Расстояние: 214.0300
Подраздел: Работа с правилами Affinity и Anti-Affinity
Источник: https://oncloud.ru/documentation/catalog/iaas/affinity
Контент: Работа с правилами Affinity и Anti-AffinityПравила Affinity и Anti-Affinity позволяют размещать несколько ВМ на разных или на одном хосте в зависимости от требований.Правило Affinity размещает несколь...

Чанк 2:
Расстояние: 249.6347
Подраздел: Как правильно использовать инфраструктуру OnCloud
Источник: https://oncloud.ru/documentation/catalog/cookbook/rekomendacii
Контент: .Более подробно можно ознакомиться тут: https://kb.vmware.com/s/article/2040375b) Рекомендуем создавать ВМ с объёмом vRAM не более 200 ГБ.При работе механизмов DRS, а также обслуживании серверов приме...

Полный промпт:
--------------------------------------------------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Ты - ассистент по документации. О

/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")



Ответ: Разница между обязательными (required) и предпочтительными (preferred) правилами Affinity и Anti-Affinity заключается в том, как они влияют на размещение виртуальных машин (ВМ):

- Обязательные правила (required):
    - ВМ подчиняются правилу, если для этого есть доступные ресурсы.
    - Если ресурсов недостаточно, правило игнорируется.
- Предпочтительные правила (preferred):
    - ВМ подчиняются правилу, если это не противоречит другим факторам.
    - Если другие факторы мешают выполнению правила, оно может быть проигнорировано.

Таким образом, обязательные правила обеспечивают строгое соблюдение условий, в то время как предпочтительные правила предоставляют гибкость и возможность учитывать дополнительные условия при размещении ВМ.
